### Obtained Perplexity= -7.47, Coherence= 0.491, Best Number of Topics = 10

In [1]:
#Basic Libraries
import re
import numpy as np
import pandas as pd
from pprint import pprint
import sys
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
#Load Dataset
df=pd.read_csv('/home/smriti/Desktop/NLP/MITACS/Anxiety/Data/CSV/Reddit/rAnxiety12.csv')
#getting rid of NaN
df=df.replace(np.nan, '', regex=True)
#getting rid of deleted values
df['Text']=df['Text'].replace('[deleted]','')
#Combining title and text
df["Post"] = df["Title"] + df["Text"]
#Now that we don't need Title or Text, we drop those columns before saving the file
df=df.drop(['Title', 'Text'], axis = 1)

/home/smriti/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [4]:
df=df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
df.head()

,Number of Comments,Date Posted,Post
0,7,2012-07-14 14:06:22 EDT-0400,The logical side of my mind in response to my ...
1,26,2012-07-10 02:14:54 EDT-0400,No matter what you're going through....
2,1,2012-07-13 23:27:10 EDT-0400,An amazing art photo you folks can probably re...
3,28,2012-07-09 01:15:00 EDT-0400,Does anyone else have the sensation that they ...
4,3,2012-07-11 04:44:15 EDT-0400,Not sure if anxious..


In [6]:
# Convert to list
data=df.Post.values.tolist()
# Remove new line characters
data=[re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data=[re.sub("\'", "", sent) for sent in data]

In [7]:
#Function to clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 

In [8]:
#Clean up text
data_words=list(sent_to_words(data))
print(data_words[:1])

[['the', 'logical', 'side', 'of', 'my', 'mind', 'in', 'response', 'to', 'my', 'emotional', 'side', 'of', 'my', 'brain', 'when', 'it', 'starts', 'being', 'anxious']]


In [9]:
# Build the bigram and trigram models
bigram=gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram=gensim.models.Phrases(bigram[data_words], threshold=100)

In [10]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod=gensim.models.phrases.Phraser(bigram)
trigram_mod=gensim.models.phrases.Phraser(trigram)

In [11]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'logical', 'side', 'of', 'my', 'mind', 'in', 'response', 'to', 'my', 'emotional', 'side', 'of', 'my', 'brain', 'when', 'it', 'starts', 'being', 'anxious']


In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [13]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

# Create Dictionary needed for topic modelling
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

[['logical', 'side', 'mind', 'response', 'emotional', 'side', 'brain', 'start', 'anxious']]


In [15]:
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.030*"round" + 0.023*"willing" + 0.015*"connect" + 0.009*"please" + '
  '0.008*"regard" + 0.008*"lurk" + 0.007*"solid" + 0.006*"negatively" + '
  '0.004*"fearful" + 0.004*"tourist"'),
 (1,
  '0.018*"acceptance" + 0.015*"cognitive_behavioral" + 0.015*"key" + '
  '0.013*"forum" + 0.010*"gut" + 0.008*"recovery" + 0.008*"mindfulness" + '
  '0.007*"relation" + 0.007*"claim" + 0.007*"available"'),
 (2,
  '0.037*"anxiety" + 0.027*"feel" + 0.024*"go" + 0.021*"be" + 0.016*"get" + '
  '0.014*"have" + 0.013*"help" + 0.011*"really" + 0.011*"start" + '
  '0.010*"would"'),
 (3,
  '0.040*"drive" + 0.038*"cop" + 0.011*"mile" + 0.010*"gas" + 0.008*"driving" '
  '+ 0.008*"unstable" + 0.007*"crappy" + 0.005*"highway" + 0.005*"nicely" + '
  '0.005*"abandon"'),
 (4,
  '0.026*"be" + 0.021*"people" + 0.020*"want" + 0.014*"know" + 0.013*"think" + '
  '0.012*"say" + 0.011*"friend" + 0.011*"go" + 0.011*"even" + 0.011*"talk"'),
 (5,
  '0.051*"relax" + 0.019*"silly" + 0.017*"large" + 0.016*"male" + '
  '

In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -7.471369644043558


In [17]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.49935489349787876


In [18]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      58.421786        1       1  0.363407 -0.001859
4      35.262797        1       2  0.355129 -0.007280
7       2.478742        1       3 -0.107814 -0.285729
6       1.178712        1       4 -0.101146  0.088353
5       1.147005        1       5 -0.104718  0.067408
9       0.674447        1       6 -0.094441  0.043778
1       0.282228        1       7 -0.079773  0.022107
8       0.238030        1       8 -0.080770  0.028268
3       0.168144        1       9 -0.074377  0.022964
0       0.148109        1      10 -0.075498  0.021990, topic_info=               Term         Freq        Total Category  logprob  loglift
term                                                                    
5736        anxiety  1675.000000  1675.000000  Default  30.0000  30.0000
1646         people   604.000000   604.000000  Default  29.0000  29.0000
3483          drive    84.000000    84.000000  Default  28.0000  28.0000
4065           want   606.000000   606.000000  Default  27.0000  27.0000
5751         friend   302.000000   302.000000  Default  26.0000  26.0000
4305           talk   282.000000   282.000000  Default  25.0000  25.0000
4330            say   379.000000   379.000000  Default  24.0000  24.0000
2649      therapist   116.000000   116.000000  Default  23.0000  23.0000
2606        breathe    49.000000    49.000000  Default  22.0000  22.0000
4793          relax    44.000000    44.000000  Default  21.0000  21.0000
98                s   248.000000   248.000000  Default  20.0000  20.0000
2485         breath    41.000000    41.000000  Default  19.0000  19.0000
3072           good   266.000000   266.000000  Default  18.0000  18.0000
4741             be  1621.000000  1621.000000  Default  17.0000  17.0000
1351          least    51.000000    51.000000  Default  16.0000  16.0000
637          slowly    40.000000    40.000000  Default  15.0000  15.0000
4903           even   436.000000   436.000000  Default  14.0000  14.0000
2653           live   146.000000   146.000000  Default  13.0000  13.0000
3867           need   311.000000   311.000000  Default  12.0000  12.0000
2252           find   282.000000   282.000000  Default  11.0000  11.0000
4405            lot   248.000000   248.000000  Default  10.0000  10.0000
1645        thought   233.000000   233.000000  Default   9.0000   9.0000
5511           call   127.000000   127.000000  Default   8.0000   8.0000
1697           feel  1378.000000  1378.000000  Default   7.0000   7.0000
3255           week   221.000000   221.000000  Default   6.0000   6.0000
4315     depression   205.000000   205.000000  Default   5.0000   5.0000
2724            bad   386.000000   386.000000  Default   4.0000   4.0000
5348            ask   160.000000   160.000000  Default   3.0000   3.0000
755         anxious   275.000000   275.000000  Default   2.0000   2.0000
5747   psychiatrist    48.000000    48.000000  Default   1.0000   1.0000
...             ...          ...          ...      ...      ...      ...
5012     negatively     0.677871     1.222220  Topic10  -5.1033   5.9255
4813        fearful     0.474899     1.019320  Topic10  -5.4592   5.7512
4958        tourist     0.418396     0.962729  Topic10  -5.5859   5.6816
5343        shortly     0.373526     0.917704  Topic10  -5.6993   5.6161
632         bounced     0.356311     0.900513  Topic10  -5.7465   5.5878
4995           liar     0.321055     0.865140  Topic10  -5.8507   5.5237
1297       avoidant     0.251679     0.795748  Topic10  -6.0941   5.3639
5597      accompany     0.233105     0.777239  Topic10  -6.1708   5.3107
4076         herbal     0.212378     0.756521  Topic10  -6.2639   5.2446
4861  confrontation     0.160353     0.704496  Topic10  -6.5449   5.0349
2283           zine     0.019453     0.588819  Topic10  -8.6543   3.1049
5233        artwork     0.019453     0.588819  Topic10  -8.6543   3.1049
1730     disclosure     

In [21]:
num_topics = 10
topic_words = []
for i in range(num_topics):
    tt = lda_model.get_topic_terms(i,10)
    topic_words.append([id2word[pair[0]] for pair in tt])

In [22]:
for i in range(0,10):
    print(topic_words[i])

['round', 'willing', 'connect', 'please', 'regard', 'lurk', 'solid', 'negatively', 'fearful', 'tourist']
['acceptance', 'cognitive_behavioral', 'key', 'forum', 'gut', 'recovery', 'mindfulness', 'relation', 'claim', 'available']
['anxiety', 'feel', 'go', 'be', 'get', 'have', 'help', 'really', 'start', 'would']
['drive', 'cop', 'mile', 'gas', 'driving', 'unstable', 'crappy', 'highway', 'nicely', 'abandon']
['be', 'people', 'want', 'know', 'think', 'say', 'friend', 'go', 'even', 'talk']
['relax', 'silly', 'large', 'male', 'psychologist', 'rush', 'flight', 'routine', 'touch', 'adult']
['breath', 'breathe', 'breathing', 'absolutely', 'around', 'leg', 'exist', 'sensation', 'major', 'middle']
['create', 'goal', 'amount', 'coffee', 'quit', 'subreddit', 'comfortable', 'note', 'least', 'cbt']
['sound', 'roll', 'chat', 'window', 'interested', 'scream', 'coyote', 'anyone_else', 'friendly', 'primal']
['visit', 'event', 'reward', 'roommate', 'coworker', 'luck', 'clear', 'cold', 'baby', 'piss']


In [25]:
df=pd.DataFrame({'Year':[],'Source':[],'Topic_ID':[],'Most_freq_words':[]})

In [26]:
df['Most_freq_words']=topic_words

In [28]:
df = df.assign(Year='2012')
df = df.assign(Source='Reddit')

In [29]:
df['Topic_ID']=[0,1,2,3,4,5,6,7,8,9]

In [30]:
df.head()

,Most_freq_words,Source,Topic_ID,Year
0,"[round, willing, connect, please, regard, lurk...",Reddit,0,2012
1,"[acceptance, cognitive_behavioral, key, forum,...",Reddit,1,2012
2,"[anxiety, feel, go, be, get, have, help, reall...",Reddit,2,2012
3,"[drive, cop, mile, gas, driving, unstable, cra...",Reddit,3,2012
4,"[be, people, want, know, think, say, friend, g...",Reddit,4,2012


In [31]:
df.to_csv("topic_words_r2012.csv")